In [55]:
import cv2
import mediapipe as mp
import time
import csv

In [69]:
class handDetector():
    def __init__(self, mode = False, maxHands = 2, modelComplexity=1, detectionCon = 0.5, trackCon = 0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.modelComplex = modelComplexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.modelComplex, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        
    def findHands(self,img, draw = True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, draw = True):
        lmdict = {}
        lmlist = []
        if self.results.multi_hand_landmarks:
            for hand_no, hand_landmarks in enumerate(self.results.multi_hand_landmarks):  
                for id, lm in enumerate(hand_landmarks.landmark):
                    h, w, c = img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    lmlist.append([id, lm.x, lm.y])
                    if draw:
                        cv2.circle(img, (cx, cy), 3, (255, 0, 255), cv2.FILLED)
            lmdict[hand_no] = lmlist
        return lmdict

def main():
    pTime = 0
    cTime = 0
    timeout = time.time() + 25
    cap = cv2.VideoCapture(0)
    detector = handDetector()
    pos = []
    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmdict = detector.findPosition(img)
        if len(lmdict) != 0:
            for key, value in lmdict.items():
                #print(f'Hand Number {key}', ' : ', value[0][1])
                for i in range(len(value)):
                    for j in range(2):
                        pos.append(value[i][j+1])          
                        
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

        cv2.imshow("Image", img)
        cv2.waitKey(1)
        #if cv2.waitKey(1) & 0xFF == ord('c'):
        if time.time() > timeout:
            break
           
    pos = list(map(list,zip(*[iter(pos)]*42))) #convert the list 'pos' to list of lists 
 
    for i in range(len(pos)):
        pos[i].insert(0,7)
        
    path = 'alphabets.csv'
    with open(path, 'a', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerows(pos)
    cap.release()
    cv2.destroyAllWindows()

In [72]:
if __name__ == "__main__":
    main()